#Setup


In [2]:
#@title ▶ Installing the required tools

!pip -q install --no-dependencies https://github.com/yaledhlab/pix-plot/archive/master.zip
!pip -q install pointgrid iiif-downloader yale-dhlab-rasterfairy yale-dhlab-keras-preprocessing
!pip -q install umap-learn==0.5.1 hdbscan

In [18]:
#@title ▶ Define the required functions

import os

from PIL import Image, ImageDraw
import json
import math
from google.colab import files

def generate_matrix(image_size, proportion_w, proportion_h, margin, background_color):
  imagelist = []
  imagelist_path = os.path.join(output_path, "data", "imagelists", f"imagelist-{output_name}.json")
  with open(imagelist_path) as imagelist_file:
    imagelist = json.loads(imagelist_file.read())

  image_sizes = imagelist["cell_sizes"][0]
  image_names = imagelist["images"]

  rasterfairy = []
  rasterfairy_path = os.path.join(output_path, "data", "layouts", f"rasterfairy-{output_name}.json")
  with open(rasterfairy_path) as rasterfairy_file:
    rasterfairy = json.loads(rasterfairy_file.read())

  width = math.ceil(math.sqrt(len(image_names)))
  height = width

  tile_size_dst_w = image_size
  tile_size_dst_h = math.floor(tile_size_dst_w * proportion_h / proportion_w)

  image_width = width * (tile_size_dst_w + 2 * margin)
  image_height = height * (tile_size_dst_h + 2 * margin)

  im = Image.new("RGB", (image_width, image_height))

  draw = ImageDraw.Draw(im)
  draw.rectangle(((0, 0), (image_width, image_height)), fill=f'#{background_color}')

  tile_w, tile_h = tile_size_dst_w, tile_size_dst_h

  for idx, filename in enumerate(image_names):
      pos = rasterfairy[idx]

      corr_pos = [((pos[0]+1)/2)*(width-1)/width, ((pos[1]+1)/2)*(height-1)/height]

      left = int(margin + corr_pos[0]*width*(tile_size_dst_w + 2 * margin))
      top = int(margin + corr_pos[1]*height*(tile_size_dst_h + 2 * margin))

      tile_im = Image.open(os.path.join(input_path, filename))
      tile_im.thumbnail((tile_size_dst_w, tile_size_dst_h), Image.ANTIALIAS)
      tile_w, tile_h = tile_im.size
      tile_disp_w = math.floor((tile_size_dst_w - tile_w)/2)
      tile_disp_h = math.floor((tile_size_dst_h - tile_h)/2)
      im.paste(tile_im, (left + tile_disp_w, top + tile_disp_h))

  output_image = os.path.join(output_path, f'{output_name}.jpg')
  im.save(output_image, quality=90)
  files.download(output_image)

  im.resize((min(1920, image_width), min(1920, image_height)))
  display(im)

## Google drive

Connect Google Colab to Google Drive to facilitate file management. This way your drive content will appear as a folder on the remote server and you can read and write files to it. Let's see how it works.

First we have to authorise the connection by executing the following code.

> There will be some warnings ⚠⚠⚠ , and there's a good reason: if you do this, the colab notebook code will have access to all your files there. In this case you can trust me 👼



In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Processing

In [ ]:
#@title ▶ Extract image features and reduce dimensionality

#@markdown This should be the path of the folder inside Drive. The best way is to find it in the file explorer on the left, click on the 3 points, use the option "copy path" and paste the value here.

#@markdown ⚠ All the images must be in jpg format.
input_path = "/content/drive/MyDrive/IAAC" #@param {type:"string"}
#@markdown The name you want to give to the UMAP (it must not contain spaces)
output_name = "IAAC" #@param {type:"string"}

input_glob = os.path.join(input_path, "*.jpg")
output_path = os.path.join("/", "content", output_name)

!pixplot --images "$input_glob" --out_dir '$output_path' --plot_id $output_name

In [ ]:
#@title 🖼 Generate image matrix

#@markdown Individual image size
image_size = 64 #@param {type:"number"}

#@markdown Individual image proportion. Leave at 1/1 if images have diferent orientations or 16/9 if they are frames from a video.
proportion_w = 1 #@param {type:"number"}
proportion_h = 1 #@param {type:"number"}

#@markdown Margin between images (in pixels)
margin = 8 #@param {type:"number"}

#@markdown Background color in hexadecimal format (you can copy/paste from photoshop). 000000 is black, ffffff is white
background_color = "000000" #@param {type:"string"}

generate_matrix(image_size, proportion_w, proportion_h, margin, background_color)